<a href="https://colab.research.google.com/github/Abhiprameesh/Case-Study-NYC-Taxi-Analytics-Platform/blob/main/Yellow_Taxi_Trip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [357 B]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,861 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [70.9 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [4,1

In [ ]:
df = spark.read.csv(
    "/content/C2_2017_Yellow_Taxi_Trip_Data.csv",
    header=True,
    inferSchema=True
)

In [ ]:
df.printSchema()


root
 |-- _c0: integer (nullable = true)
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [ ]:
df.show(5)
df.count()

+---------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|      _c0|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+---------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
| 24870114|       2|03/25/2017 8:55:4...| 03/25/2017 9:09:4...|              6|         3.34|         1|                 N|         100|         231|           1|       13.0|  0.0|    0.5|      2.76|         0.0|                  0.3|       16.56|
| 356342

22699

In [ ]:
df.describe(["trip_distance", "total_amount"]).show()

+-------+------------------+------------------+
|summary|     trip_distance|      total_amount|
+-------+------------------+------------------+
|  count|             22699|             22699|
|   mean|2.9133129212740796| 16.31050222476416|
| stddev|3.6531711828339195|16.097295300636247|
|    min|               0.0|            -120.3|
|    max|             33.96|           1200.29|
+-------+------------------+------------------+



In [ ]:
df.summary().show()

+-------+--------------------+------------------+--------------------+---------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+------------------+---------------------+------------------+
|summary|                 _c0|          VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|   passenger_count|     trip_distance|        RatecodeID|store_and_fwd_flag|      PULocationID|      DOLocationID|       payment_type|       fare_amount|              extra|            mta_tax|        tip_amount|      tolls_amount|improvement_surcharge|      total_amount|
+-------+--------------------+------------------+--------------------+---------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+-----

In [ ]:
df.select("passenger_count").distinct().show()


+---------------+
|passenger_count|
+---------------+
|              1|
|              6|
|              3|
|              5|
|              4|
|              2|
|              0|
+---------------+



In [ ]:
df_no_duplicates = df.dropDuplicates()

In [ ]:
df.dropDuplicates(["tpep_pickup_datetime", "PULocationID"])

DataFrame[_c0: int, VendorID: int, tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: int, trip_distance: double, RatecodeID: int, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: int, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double]

In [ ]:
clean_df = df.filter(
    (df.trip_distance > 0) &
    (df.total_amount > 0)
)

In [ ]:
clean_df = df.where("trip_distance > 0 AND total_amount > 0")

In [ ]:
clean_df.filter(clean_df.passenger_count.isNull()).count()

0

In [ ]:
clean_df = clean_df.dropna(subset=["passenger_count"])

In [ ]:
from pyspark.sql.functions import col

clean_df = clean_df.withColumn(
    "revenue_per_km",
    col("total_amount") / (col("trip_distance") * 1.6)
)

In [ ]:
from pyspark.sql.functions import sum
from pyspark.sql.functions import count
from pyspark.sql.functions import avg

clean_df.select(sum("total_amount")).show()

+-----------------+
|sum(total_amount)|
+-----------------+
|366331.9999999227|
+-----------------+



In [ ]:
clean_df.groupBy("passenger_count") \
    .agg(
        sum("total_amount").alias("total_revenue"),
        avg("trip_distance").alias("avg_distance"),
        count("*").alias("trip_count")
    ) \
    .show()

+---------------+------------------+------------------+----------+
|passenger_count|     total_revenue|      avg_distance|trip_count|
+---------------+------------------+------------------+----------+
|              1|256639.67999996367| 2.860989306484904|     15991|
|              6|11129.749999999973| 3.027267441860466|       688|
|              3| 15732.28999999989|3.0660168598524757|       949|
|              5|18595.189999999773|2.9472154115586684|      1142|
|              4|7421.5000000000355|2.9916666666666654|       450|
|              2| 56309.99000000148|3.2209010654490013|      3285|
|              0|503.60000000000014|2.6093749999999996|        32|
+---------------+------------------+------------------+----------+



In [ ]:
from pyspark.sql.functions import hour, to_timestamp, col

# Convert string datetime (with AM/PM) into proper timestamp
clean_df = clean_df.withColumn(
    "pickup_time",
    to_timestamp(col("tpep_pickup_datetime"), "MM/dd/yyyy h:mm:ss a")
)

# Extract hour from timestamp (0–23)
clean_df = clean_df.withColumn(
    "pickup_hour",
    hour(col("pickup_time"))
)

# Count number of pickups per hour and sort
clean_df.groupBy("pickup_hour") \
    .count() \
    .orderBy("pickup_hour") \
    .show()

+-----------+-----+
|pickup_hour|count|
+-----------+-----+
|          0|  730|
|          1|  521|
|          2|  382|
|          3|  284|
|          4|  231|
|          5|  234|
|          6|  495|
|          7|  850|
|          8| 1012|
|          9| 1102|
|         10| 1035|
|         11| 1085|
|         12| 1106|
|         13| 1120|
|         14| 1220|
|         15| 1163|
|         16| 1121|
|         17| 1208|
|         18| 1440|
|         19| 1445|
+-----------+-----+
only showing top 20 rows



In [ ]:
# Import required functions
from pyspark.sql.functions import hour, to_timestamp, col


clean_df = clean_df.withColumn(
    "pickup_time",
    to_timestamp(col("tpep_pickup_datetime"), "yyyy-MM-dd HH:mm:ss")
)


clean_df = clean_df.withColumn(
    "pickup_hour",
    hour(col("pickup_time"))
)
clean_df.groupBy("pickup_hour") \
    .count() \
    .orderBy("pickup_hour") \
    .show()
clean_df.filter(col("pickup_time").isNull()).show(truncate=False)

+-----------+-----+
|pickup_hour|count|
+-----------+-----+
|       null|22537|
+-----------+-----+

+---------+--------+----------------------+----------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------------+-----------+-----------+
|_c0      |VendorID|tpep_pickup_datetime  |tpep_dropoff_datetime |passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|revenue_per_km    |pickup_time|pickup_hour|
+---------+--------+----------------------+----------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------------+-----------+-----------+
|24870114 |2     

Task1


Task 2



In [ ]:
from pyspark.sql.functions import col

# Create the revenue_per_mile column
clean_df = clean_df.withColumn(
    "revenue_per_mile",
    col("total_amount") / col("trip_distance")
)

print("Added 'revenue_per_mile' column.")

Added 'revenue_per_mile' column.


In [ ]:
from pyspark.sql.functions import avg

# Compute average revenue per mile
avg_revenue_per_mile = clean_df.select(avg("revenue_per_mile").alias("average_revenue_per_mile"))

print("Average Revenue per Mile:")
avg_revenue_per_mile.show()

Average Revenue per Mile:
+------------------------+
|average_revenue_per_mile|
+------------------------+
|       9.475535080491763|
+------------------------+



In [ ]:
# Top 15 trips with highest revenue per mile
print("Top 15 Trips with Highest Revenue per Mile:")
clean_df.orderBy(col("revenue_per_mile").desc()).limit(15).show()

Top 15 Trips with Highest Revenue per Mile:
+---------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------------+-------------------+-----------+------------------+
|      _c0|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|    revenue_per_km|        pickup_time|pickup_hour|  revenue_per_mile|
+---------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------------+-------------------+-----------+------------------+
|1

### Task 1: Distance Insights



In [ ]:
# Re-creating clean_df as it was not defined in the previous execution context.
# Filter out rows with trip_distance or total_amount less than or equal to 0
clean_df = df.filter(
    (df.trip_distance > 0) &
    (df.total_amount > 0)
)

In [ ]:
# Drop rows where 'passenger_count' is null
clean_df = clean_df.dropna(subset=["passenger_count"])

In [ ]:
from pyspark.sql.functions import col

# Add 'revenue_per_km' column
clean_df = clean_df.withColumn(
    "revenue_per_km",
    col("total_amount") / (col("trip_distance") * 1.6)
)

In [ ]:
from pyspark.sql.functions import hour, to_timestamp

# Convert string datetime (with AM/PM) into proper timestamp
clean_df = clean_df.withColumn(
    "pickup_time",
    to_timestamp(col("tpep_pickup_datetime"), "MM/dd/yyyy h:mm:ss a")
)

# Extract hour from timestamp (0–23)
clean_df = clean_df.withColumn(
    "pickup_hour",
    hour(col("pickup_time"))
)

In [ ]:
from pyspark.sql.functions import min, max, avg, percentile_approx

# Calculate distance statistics
distance_stats = clean_df.select(
    min("trip_distance").alias("min_trip_distance"),
    max("trip_distance").alias("max_trip_distance"),
    avg("trip_distance").alias("avg_trip_distance"),
    percentile_approx("trip_distance", 0.5, 100).alias("median_trip_distance")
)

# Display the results
distance_stats.show()

+-----------------+-----------------+------------------+--------------------+
|min_trip_distance|max_trip_distance| avg_trip_distance|median_trip_distance|
+-----------------+-----------------+------------------+--------------------+
|             0.01|            33.96|2.9337808936415817|                 1.6|
+-----------------+-----------------+------------------+--------------------+



### Task 3: Passenger Demand Pattern



In [ ]:
from pyspark.sql.functions import sum, avg, count

# Calculate passenger performance metrics
passenger_performance = clean_df.groupBy("passenger_count") \
    .agg(
        count("*").alias("total_trips"),
        avg("trip_distance").alias("average_trip_distance"),
        sum("total_amount").alias("total_revenue")
    ) \
    .orderBy("passenger_count")

# Display the results
print("Passenger Performance Table:")
passenger_performance.show()

Passenger Performance Table:
+---------------+-----------+---------------------+------------------+
|passenger_count|total_trips|average_trip_distance|     total_revenue|
+---------------+-----------+---------------------+------------------+
|              0|         32|   2.6093749999999996|503.60000000000014|
|              1|      15991|    2.860989306484904|256639.67999996367|
|              2|       3285|   3.2209010654490013| 56309.99000000148|
|              3|        949|   3.0660168598524757| 15732.28999999989|
|              4|        450|   2.9916666666666654|7421.5000000000355|
|              5|       1142|   2.9472154115586684|18595.189999999773|
|              6|        688|    3.027267441860466|11129.749999999973|
+---------------+-----------+---------------------+------------------+



### Task 4: Pickup Time Analytics



In [ ]:
from pyspark.sql.functions import dayofweek

# Extract weekday from timestamp (1 = Sunday, 7 = Saturday)
clean_df = clean_df.withColumn(
    "pickup_weekday",
    dayofweek(col("pickup_time"))
)

print("Added 'pickup_weekday' column.")

Added 'pickup_weekday' column.


In [ ]:
from pyspark.sql.functions import count

# Count trips per hour
trips_per_hour = clean_df.groupBy("pickup_hour") \
    .agg(count("*").alias("total_trips")) \
    .orderBy("pickup_hour")

print("Trips per Hour:")
trips_per_hour.show()

Trips per Hour:
+-----------+-----------+
|pickup_hour|total_trips|
+-----------+-----------+
|          0|        730|
|          1|        521|
|          2|        382|
|          3|        284|
|          4|        231|
|          5|        234|
|          6|        495|
|          7|        850|
|          8|       1012|
|          9|       1102|
|         10|       1035|
|         11|       1085|
|         12|       1106|
|         13|       1120|
|         14|       1220|
|         15|       1163|
|         16|       1121|
|         17|       1208|
|         18|       1440|
|         19|       1445|
+-----------+-----------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import avg

# Compute average revenue per weekday
avg_revenue_per_weekday = clean_df.groupBy("pickup_weekday") \
    .agg(avg("total_amount").alias("average_revenue")) \
    .orderBy("pickup_weekday")

print("Average Revenue per Weekday:")
avg_revenue_per_weekday.show()

Average Revenue per Weekday:
+--------------+------------------+
|pickup_weekday|   average_revenue|
+--------------+------------------+
|             1| 16.08965725806481|
|             2|16.873152658662416|
|             3|16.144403032217614|
|             4|16.334314540059673|
|             5|16.785422723227946|
|             6|16.396767050487565|
|             7|15.210205847255702|
+--------------+------------------+



### Task 5: Trip Duration Engineering



In [ ]:
from pyspark.sql.functions import col, to_timestamp

# Convert dropoff datetime string to timestamp
clean_df = clean_df.withColumn(
    "dropoff_time",
    to_timestamp(col("tpep_dropoff_datetime"), "MM/dd/yyyy h:mm:ss a")
)

# Calculate trip duration in minutes
# (dropoff_time - pickup_time) gives an interval, cast to long to get seconds, then divide by 60
clean_df = clean_df.withColumn(
    "trip_duration_minutes",
    (col("dropoff_time").cast("long") - col("pickup_time").cast("long")) / 60
)

print("Added 'trip_duration_minutes' column.")

Added 'trip_duration_minutes' column.


In [ ]:
from pyspark.sql.functions import avg

# Compute average duration
avg_duration = clean_df.select(avg("trip_duration_minutes").alias("average_trip_duration_minutes"))

print("Average Trip Duration:")
avg_duration.show()

Average Trip Duration:
+-----------------------------+
|average_trip_duration_minutes|
+-----------------------------+
|           17.064017689429182|
+-----------------------------+



In [ ]:
from pyspark.sql.functions import desc

# Longest 10 trips
print("Top 10 Longest Trips (by duration):")
clean_df.orderBy(desc("trip_duration_minutes")).limit(10).show()

Top 10 Longest Trips (by duration):
+---------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------------+-------------------+-----------+------------------+--------------+-------------------+---------------------+
|      _c0|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|    revenue_per_km|        pickup_time|pickup_hour|  revenue_per_mile|pickup_weekday|       dropoff_time|trip_duration_minutes|
+---------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+--

In [ ]:
from pyspark.sql.functions import count

# Trips shorter than 2 minutes (data anomaly check)
short_trips_count = clean_df.filter(col("trip_duration_minutes") < 2).count()

print(f"Number of trips shorter than 2 minutes: {short_trips_count}")

Number of trips shorter than 2 minutes: 383


In [ ]:
import os
if not os.path.exists("/content/taxi+_zone_lookup.csv"):
    !wget -q https://raw.githubusercontent.com/databricks-academy/data-engineering-with-databricks-2023/main/retail-org/taxi_zone_lookup.csv -O /content/taxi+_zone_lookup.csv
zone_df = spark.read.csv(
    "/content/taxi+_zone_lookup.csv",
    header=True,
    inferSchema=True
)
print("Zone lookup data loaded into zone_df.")

Zone lookup data loaded into zone_df.


In [ ]:
from pyspark.sql.functions import col, count, lit

# Count total trips to calculate percentages later
total_trips = clean_df_with_borough.count()

# Group by 'Borough', count trips, and calculate the proportion
borough_trip_proportions = clean_df_with_borough.groupBy("Borough") \
    .agg(count("*").alias("trip_count")) \
    .withColumn("proportion", col("trip_count") / lit(total_trips) * 100) \
    .orderBy(col("trip_count").desc())

print("Proportion of Trips per Borough:")
borough_trip_proportions.show(truncate=False)

Proportion of Trips per Borough:
+-------+----------+----------+
|Borough|trip_count|proportion|
+-------+----------+----------+
|null   |22537     |100.0     |
+-------+----------+----------+



In [ ]:
print("Schema of zone_df:")
zone_df.printSchema()

print("First 5 rows of zone_df:")
zone_df.show(5, truncate=False)

Schema of zone_df:
root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

First 5 rows of zone_df:
+----------+-------+----+------------+
|LocationID|Borough|Zone|service_zone|
+----------+-------+----+------------+
+----------+-------+----+------------+



In [ ]:
print(f"Count of rows in zone_df: {zone_df.count()}")

print("Content of taxi_zone_lookup_renamed.csv:")
!head /content/taxi_zone_lookup_renamed.csv

Count of rows in zone_df: 0
Content of taxi_zone_lookup_renamed.csv:


In [ ]:
import os

original_file_path = "/content/taxi+_zone_lookup.csv"
renamed_file_path = "/content/taxi_zone_lookup_renamed.csv"

# Remove the renamed file if it exists and is empty/corrupted
if os.path.exists(renamed_file_path):
    os.remove(renamed_file_path)
    print(f"Removed existing file: {renamed_file_path}")

# Re-download the file
!wget -q https://raw.githubusercontent.com/databricks-academy/data-engineering-with-databricks-2023/main/retail-org/taxi_zone_lookup.csv -O "$original_file_path"
print(f"Re-downloaded file to: {original_file_path}")

# Rename the file again
if os.path.exists(original_file_path) and not os.path.exists(renamed_file_path):
    !mv "$original_file_path" "$renamed_file_path"
    print(f"File renamed from {original_file_path} to {renamed_file_path}")

print("File download and renaming complete.")

Removed existing file: /content/taxi_zone_lookup_renamed.csv
Re-downloaded file to: /content/taxi+_zone_lookup.csv
File renamed from /content/taxi+_zone_lookup.csv to /content/taxi_zone_lookup_renamed.csv
File download and renaming complete.


In [ ]:
import os
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define the original and new file paths
original_file_path = "/content/taxi+_zone_lookup.csv"
renamed_file_path = "/content/taxi_zone_lookup_renamed.csv"

# Define the schema explicitly for the zone lookup file
zone_schema = StructType([
    StructField("LocationID", IntegerType(), True),
    StructField("Borough", StringType(), True),
    StructField("Zone", StringType(), True),
    StructField("service_zone", StringType(), True)
])

# Load zone_df using the renamed file and explicit schema
zone_df = spark.read.csv(
    renamed_file_path,
    header=True,
    schema=zone_schema # Use explicit schema instead of inferSchema=True
)

# Join clean_df with zone_df on PULocationID
clean_df_with_borough = clean_df.join(
    zone_df.withColumnRenamed("LocationID", "PULocationID"),
    on="PULocationID",
    how="left"
)

print("Joined taxi trip data with zone lookup data to add borough information.")
clean_df_with_borough.printSchema()
clean_df_with_borough.show(5, truncate=False)


Joined taxi trip data with zone lookup data to add borough information.
root
 |-- PULocationID: integer (nullable = true)
 |-- _c0: integer (nullable = true)
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- revenue_per_km: double (nullable = true)
 |-- pickup_time: timestamp (nullable = true)
 |-- pickup_hour: integer 

In [ ]:
print("Schema of zone_df after loading with explicit schema:")
zone_df.printSchema()

print("First 5 rows of zone_df after loading with explicit schema:")
zone_df.show(5, truncate=False)

print(f"Count of rows in zone_df: {zone_df.count()}")

Schema of zone_df after loading with explicit schema:
root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

First 5 rows of zone_df after loading with explicit schema:
+----------+-------+----+------------+
|LocationID|Borough|Zone|service_zone|
+----------+-------+----+------------+
+----------+-------+----+------------+

Count of rows in zone_df: 0


In [ ]:
print("Verifying content of taxi_zone_lookup_renamed.csv:")
!ls -lh /content/taxi_zone_lookup_renamed.csv
!head /content/taxi_zone_lookup_renamed.csv

Verifying content of taxi_zone_lookup_renamed.csv:
-rw-r--r-- 1 root root 0 Feb 26 06:46 /content/taxi_zone_lookup_renamed.csv


In [ ]:
import os
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

original_file_path = "/content/taxi+_zone_lookup.csv"
renamed_file_path = "/content/taxi_zone_lookup_renamed.csv"

# Ensure the old, empty file is removed before trying to download again
if os.path.exists(renamed_file_path) and os.path.getsize(renamed_file_path) == 0:
    os.remove(renamed_file_path)
    print(f"Removed empty file: {renamed_file_path}")

# Re-download the file. Removed -q to potentially see errors.
# Remove the old original file if it exists to ensure a fresh download
if os.path.exists(original_file_path):
    os.remove(original_file_path)
!wget https://raw.githubusercontent.com/databricks-academy/data-engineering-with-databricks-2023/main/retail-org/taxi_zone_lookup.csv -O "$original_file_path"
print(f"Re-downloaded file to: {original_file_path}")

# Rename the file again
if os.path.exists(original_file_path):
    if os.path.exists(renamed_file_path):
        os.remove(renamed_file_path) # Remove if already exists to prevent mv error
    !mv "$original_file_path" "$renamed_file_path"
    print(f"File renamed from {original_file_path} to {renamed_file_path}")

# Verify if the downloaded file is not empty
if not os.path.exists(renamed_file_path) or os.path.getsize(renamed_file_path) == 0:
    print(f"Error: The downloaded file {renamed_file_path} is still empty. Cannot proceed with loading data.")
    # The subtask cannot be completed successfully without the data
else:
    print(f"File {renamed_file_path} successfully downloaded and verified (size: {os.path.getsize(renamed_file_path)} bytes).")

    # Define the schema explicitly for the zone lookup file
    zone_schema = StructType([
        StructField("LocationID", IntegerType(), True),
        StructField("Borough", StringType(), True),
        StructField("Zone", StringType(), True),
        StructField("service_zone", StringType(), True)
    ])

    # Load zone_df using the renamed file and explicit schema
    zone_df = spark.read.csv(
        renamed_file_path,
        header=True,
        schema=zone_schema # Use explicit schema instead of inferSchema=True
    )

    # Join clean_df with zone_df on PULocationID
    clean_df_with_borough = clean_df.join(
        zone_df.withColumnRenamed("LocationID", "PULocationID"),
        on="PULocationID",
        how="left"
    )

    print("Joined taxi trip data with zone lookup data to add borough information.")
    clean_df_with_borough.printSchema()
    clean_df_with_borough.show(5, truncate=False)


Removed empty file: /content/taxi_zone_lookup_renamed.csv
--2026-02-26 06:47:45--  https://raw.githubusercontent.com/databricks-academy/data-engineering-with-databricks-2023/main/retail-org/taxi_zone_lookup.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2026-02-26 06:47:45 ERROR 404: Not Found.

Re-downloaded file to: /content/taxi+_zone_lookup.csv
File renamed from /content/taxi+_zone_lookup.csv to /content/taxi_zone_lookup_renamed.csv
Error: The downloaded file /content/taxi_zone_lookup_renamed.csv is still empty. Cannot proceed with loading data.


In [ ]:
import os
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

original_download_path = "/content/taxi_zone_lookup.csv"
renamed_file_path = "/content/taxi_zone_lookup_renamed.csv"

# 1. Remove any existing files to ensure a clean download
if os.path.exists(original_download_path):
    os.remove(original_download_path)
    print(f"Removed existing file: {original_download_path}")
if os.path.exists(renamed_file_path):
    os.remove(renamed_file_path)
    print(f"Removed existing file: {renamed_file_path}")

# 2. Download the file from the corrected URL
new_url = "https://raw.githubusercontent.com/toddwschneider/nyc-taxi-data/master/taxi_zone_lookup.csv"
!wget "{new_url}" -O "{original_download_path}"
print(f"Downloaded file from {new_url} to: {original_download_path}")

# 3. Rename the downloaded file
if os.path.exists(original_download_path):
    os.rename(original_download_path, renamed_file_path)
    print(f"File renamed from {original_download_path} to {renamed_file_path}")
else:
    print(f"Error: Original downloaded file {original_download_path} not found.")

# 4. Verify that the file exists and is not empty
if not os.path.exists(renamed_file_path) or os.path.getsize(renamed_file_path) == 0:
    print(f"Error: The downloaded file {renamed_file_path} is empty or does not exist. Cannot proceed with loading data.")
else:
    print(f"File {renamed_file_path} successfully downloaded and verified (size: {os.path.getsize(renamed_file_path)} bytes).\n")

    # 5. Define the zone_schema explicitly
    zone_schema = StructType([
        StructField("LocationID", IntegerType(), True),
        StructField("Borough", StringType(), True),
        StructField("Zone", StringType(), True),
        StructField("service_zone", StringType(), True)
    ])

    # 6. Load zone_df using the renamed file and explicit schema
    zone_df = spark.read.csv(
        renamed_file_path,
        header=True,
        schema=zone_schema
    )

    # 7. Print the schema of zone_df and show its first 5 rows
    print("Schema of zone_df:")
    zone_df.printSchema()
    print("\nFirst 5 rows of zone_df:")
    zone_df.show(5, truncate=False)

    # 8. Print the total count of rows in zone_df
    print(f"\nTotal count of rows in zone_df: {zone_df.count()}")

Removed existing file: /content/taxi_zone_lookup_renamed.csv
--2026-02-26 06:48:28--  https://raw.githubusercontent.com/toddwschneider/nyc-taxi-data/master/taxi_zone_lookup.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2026-02-26 06:48:28 ERROR 404: Not Found.

Downloaded file from https://raw.githubusercontent.com/toddwschneider/nyc-taxi-data/master/taxi_zone_lookup.csv to: /content/taxi_zone_lookup.csv
File renamed from /content/taxi_zone_lookup.csv to /content/taxi_zone_lookup_renamed.csv
Error: The downloaded file /content/taxi_zone_lookup_renamed.csv is empty or does not exist. Cannot proceed with loading data.


In [ ]:
import os
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

original_download_path = "/content/taxi_zone_lookup.csv"
renamed_file_path = "/content/taxi_zone_lookup_renamed.csv"

# 1. Remove any existing files to ensure a clean download
if os.path.exists(original_download_path):
    os.remove(original_download_path)
    print(f"Removed existing file: {original_download_path}")
if os.path.exists(renamed_file_path):
    os.remove(renamed_file_path)
    print(f"Removed existing file: {renamed_file_path}")

# 2. Download the file from the corrected URL
# Found a working URL from a similar dataset source
new_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"
!wget "{new_url}" -O "{original_download_path}"
print(f"Downloaded file from {new_url} to: {original_download_path}")

# 3. Rename the downloaded file
if os.path.exists(original_download_path):
    os.rename(original_download_path, renamed_file_path)
    print(f"File renamed from {original_download_path} to {renamed_file_path}")
else:
    print(f"Error: Original downloaded file {original_download_path} not found.")

# 4. Verify that the file exists and is not empty
if not os.path.exists(renamed_file_path) or os.path.getsize(renamed_file_path) == 0:
    print(f"Error: The downloaded file {renamed_file_path} is empty or does not exist. Cannot proceed with loading data.")
else:
    print(f"File {renamed_file_path} successfully downloaded and verified (size: {os.path.getsize(renamed_file_path)} bytes).\n")

    # 5. Define the zone_schema explicitly
    zone_schema = StructType([
        StructField("LocationID", IntegerType(), True),
        StructField("Borough", StringType(), True),
        StructField("Zone", StringType(), True),
        StructField("service_zone", StringType(), True)
    ])

    # 6. Load zone_df using the renamed file and explicit schema
    zone_df = spark.read.csv(
        renamed_file_path,
        header=True,
        schema=zone_schema
    )

    # 7. Print the schema of zone_df and show its first 5 rows
    print("Schema of zone_df:")
    zone_df.printSchema()
    print("\nFirst 5 rows of zone_df:")
    zone_df.show(5, truncate=False)

    # 8. Print the total count of rows in zone_df
    print(f"\nTotal count of rows in zone_df: {zone_df.count()}")

Removed existing file: /content/taxi_zone_lookup_renamed.csv
--2026-02-26 06:48:46--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.38.181, 18.239.38.147, 18.239.38.163, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.38.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘/content/taxi_zone_lookup.csv’

/content/taxi_zone_ 100%[===================>]  12.04K  --.-KB/s    in 0s      

2026-02-26 06:48:46 (193 MB/s) - ‘/content/taxi_zone_lookup.csv’ saved [12331/12331]

Downloaded file from https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv to: /content/taxi_zone_lookup.csv
File renamed from /content/taxi_zone_lookup.csv to /content/taxi_zone_lookup_renamed.csv
File /content/taxi_zone_lookup_renamed.csv successfully downloaded and verified (size: 12331 bytes).

Schema of zone_df:

In [ ]:
from pyspark.sql.functions import col

# Rename 'LocationID' in zone_df to 'PULocationID' to match clean_df
zone_df_renamed = zone_df.withColumnRenamed("LocationID", "PULocationID")

# Perform a left join operation between clean_df and the renamed zone_df
clean_df_with_borough = clean_df.join(
    zone_df_renamed,
    on="PULocationID",
    how="left"
)

# Display the schema of the new clean_df_with_borough DataFrame
print("Schema of clean_df_with_borough:")
clean_df_with_borough.printSchema()

# Show the first 5 rows of the clean_df_with_borough DataFrame to verify the join
print("\nFirst 5 rows of clean_df_with_borough:")
clean_df_with_borough.show(5, truncate=False)

Schema of clean_df_with_borough:
root
 |-- PULocationID: integer (nullable = true)
 |-- _c0: integer (nullable = true)
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- revenue_per_km: double (nullable = true)
 |-- pickup_time: timestamp (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- revenue_per_mile

## Compute Total Revenue per Borough

### Subtask:
Group the joined DataFrame by 'Borough' and calculate the sum of 'total_amount' to determine the total revenue for each Borough.


In [ ]:
from pyspark.sql.functions import sum, col

# Group by 'Borough' and calculate the sum of 'total_amount'
total_revenue_per_borough = clean_df_with_borough.groupBy("Borough") \
    .agg(sum("total_amount").alias("total_revenue")) \
    .orderBy(col("total_revenue").desc())

# Display the results
print("Total Revenue per Borough:")
total_revenue_per_borough.show(truncate=False)

Total Revenue per Borough:
+-------------+------------------+
|Borough      |total_revenue     |
+-------------+------------------+
|Manhattan    |293290.699999942  |
|Queens       |61479.07000000016 |
|Unknown      |5578.510000000022 |
|Brooklyn     |5308.000000000016 |
|Bronx        |258.93000000000006|
|N/A          |257.19            |
|EWR          |152.3             |
|Staten Island|7.3               |
+-------------+------------------+



## Compute Average Trip Distance per Borough

### Subtask:
Group the joined DataFrame by 'Borough' and calculate the average of 'trip_distance' to find the average trip distance for each Borough.


In [ ]:
from pyspark.sql.functions import avg, col

# Group by 'Borough' and calculate the average of 'trip_distance'
average_trip_distance_per_borough = clean_df_with_borough.groupBy("Borough") \
    .agg(avg("trip_distance").alias("average_trip_distance")) \
    .orderBy("Borough")

# Display the results
print("Average Trip Distance per Borough:")
average_trip_distance_per_borough.show(truncate=False)

Average Trip Distance per Borough:
+-------------+---------------------+
|Borough      |average_trip_distance|
+-------------+---------------------+
|Bronx        |3.8593750000000004   |
|Brooklyn     |3.4224358974358995   |
|EWR          |7.3                  |
|Manhattan    |2.3759209050568213   |
|N/A          |4.227142857142857    |
|Queens       |11.148119970738836   |
|Staten Island|1.43                 |
|Unknown      |3.031472392638033    |
+-------------+---------------------+



## Display Borough Performance Table




In [ ]:
from pyspark.sql.functions import col

# Join the total_revenue_per_borough DataFrame with the average_trip_distance_per_borough DataFrame
borough_performance_table = total_revenue_per_borough.join(
    average_trip_distance_per_borough,
    on="Borough",
    how="inner"
).orderBy("Borough")

# Display the borough_performance_table DataFrame
print("Borough Performance Table:")
borough_performance_table.show(truncate=False)

Borough Performance Table:
+-------------+------------------+---------------------+
|Borough      |total_revenue     |average_trip_distance|
+-------------+------------------+---------------------+
|Bronx        |258.93000000000006|3.8593750000000004   |
|Brooklyn     |5308.000000000016 |3.4224358974358995   |
|EWR          |152.3             |7.3                  |
|Manhattan    |293290.699999942  |2.3759209050568213   |
|N/A          |257.19            |4.227142857142857    |
|Queens       |61479.07000000016 |11.148119970738836   |
|Staten Island|7.3               |1.43                 |
|Unknown      |5578.510000000022 |3.031472392638033    |
+-------------+------------------+---------------------+



## Top 10 Pickup Zones by Trips



In [ ]:
from pyspark.sql.functions import col, count

# Group by 'Zone', count trips, and order the results
top_10_pickup_zones = clean_df_with_borough.groupBy("Zone") \
    .agg(count("*").alias("total_pickups")) \
    .orderBy(col("total_pickups").desc()) \
    .limit(10)

# Display the results
print("Top 10 Pickup Zones by Trips:")
top_10_pickup_zones.show(truncate=False)

Top 10 Pickup Zones by Trips:
+----------------------------+-------------+
|Zone                        |total_pickups|
+----------------------------+-------------+
|Upper East Side South       |888          |
|Midtown Center              |858          |
|Penn Station/Madison Sq West|786          |
|Upper East Side North       |784          |
|Midtown East                |771          |
|Union Sq                    |746          |
|Murray Hill                 |745          |
|Clinton East                |740          |
|Times Sq/Theatre District   |735          |
|Lincoln Square East         |647          |
+----------------------------+-------------+



## Top 10 Dropoff Zones by Revenue




**Reasoning**:
The previous code failed due to an `IndentationError`. I need to correct the indentation for the `.limit(10)` method call to properly chain it with the `.orderBy()` method.



**Reasoning**:
The previous code consistently failed due to an `IndentationError` where `.limit(10)` was on a new, incorrectly indented line. I will correct this by placing `.limit(10)` on the same line as `.orderBy(col("total_revenue").desc())` to ensure proper method chaining.



In [ ]:
from pyspark.sql.functions import sum, col

# 1. Group by 'DOLocationID' and calculate the sum of 'total_amount'
dropoff_revenue_df = clean_df_with_borough.groupBy("DOLocationID") \
    .agg(sum("total_amount").alias("total_revenue"))

# 2. Join with zone_df_renamed to get the dropoff zone names
top_10_dropoff_zones = dropoff_revenue_df.join(
    zone_df_renamed.withColumnRenamed("PULocationID", "DOLocationID"),
    on="DOLocationID",
    how="inner"
)

# 3. Order the resulting DataFrame by 'total_revenue' in descending order
# 4. Select the top 10 rows from the ordered DataFrame
top_10_dropoff_zones = top_10_dropoff_zones.orderBy(col("total_revenue").desc()).limit(10)

# 5. Display the results
print("Top 10 Dropoff Zones by Revenue:")
top_10_dropoff_zones.show(truncate=False)

Top 10 Dropoff Zones by Revenue:
+------------+------------------+---------+----------------------------+------------+
|DOLocationID|total_revenue     |Borough  |Zone                        |service_zone|
+------------+------------------+---------+----------------------------+------------+
|230         |12676.199999999915|Manhattan|Times Sq/Theatre District   |Yellow Zone |
|132         |12447.010000000007|Queens   |JFK Airport                 |Airports    |
|161         |12306.129999999943|Manhattan|Midtown Center              |Yellow Zone |
|138         |10753.03          |Queens   |LaGuardia Airport           |Airports    |
|162         |10112.499999999993|Manhattan|Midtown East                |Yellow Zone |
|236         |9514.820000000014 |Manhattan|Upper East Side North       |Yellow Zone |
|48          |9286.47000000001  |Manhattan|Clinton East                |Yellow Zone |
|170         |9098.60000000003  |Manhattan|Murray Hill                 |Yellow Zone |
|186         |8673.65

In [ ]:
from pyspark.sql.functions import when, col

# Add the 'distance_category' column
clean_df_with_borough = clean_df_with_borough.withColumn(
    "distance_category",
    when(col("trip_distance") < 1, "Micro")
    .when((col("trip_distance") >= 1) & (col("trip_distance") < 3), "Short")
    .when((col("trip_distance") >= 3) & (col("trip_distance") < 10), "Medium")
    .otherwise("Long")
)

print("Added 'distance_category' column to clean_df_with_borough.")
clean_df_with_borough.select("trip_distance", "distance_category").show(5)
clean_df_with_borough.groupBy("distance_category").count().show()

Added 'distance_category' column to clean_df_with_borough.
+-------------+-----------------+
|trip_distance|distance_category|
+-------------+-----------------+
|         3.34|           Medium|
|          1.8|            Short|
|          1.0|            Short|
|          3.7|           Medium|
|         4.37|           Medium|
+-------------+-----------------+
only showing top 5 rows

+-----------------+-----+
|distance_category|count|
+-----------------+-----+
|           Medium| 4544|
|            Micro| 5526|
|             Long| 1312|
|            Short|11155|
+-----------------+-----+



In [ ]:
from pyspark.sql.functions import sum, avg, count

# Group by 'distance_category' and calculate the required metrics
distance_category_performance = clean_df_with_borough.groupBy("distance_category") \
    .agg(
        count("*").alias("total_trips"),
        avg("total_amount").alias("average_total_revenue"),
        avg("trip_duration_minutes").alias("average_trip_duration_minutes")
    ) \
    .orderBy("distance_category")

# Display the results
print("Distance Category Performance Table:")
distance_category_performance.show(truncate=False)

Distance Category Performance Table:
+-----------------+-----------+---------------------+-----------------------------+
|distance_category|total_trips|average_total_revenue|average_trip_duration_minutes|
+-----------------+-----------+---------------------+-----------------------------+
|Long             |1312       |57.0151753048781     |44.744258130081285           |
|Medium           |4544       |24.5552310739446     |27.5708993544601             |
|Micro            |5526       |7.686825913861883    |7.761693207865856            |
|Short            |11155      |12.323775885254      |14.136621843717345           |
+-----------------+-----------+---------------------+-----------------------------+



### Task 9: Payment Behavior Insights



In [ ]:
from pyspark.sql.functions import sum, avg, count, col, lit

# Calculate total trips for percentage calculation
total_trips_all_types = clean_df_with_borough.count()

# Group by 'payment_type' and calculate the required metrics
payment_behavior = clean_df_with_borough.groupBy("payment_type") \
    .agg(
        count("*").alias("total_trips"),
        avg("total_amount").alias("average_revenue"),
        sum("total_amount").alias("total_revenue")
    ) \
    .withColumn("percentage_contribution", (col("total_trips") / lit(total_trips_all_types)) * 100) \
    .orderBy("payment_type")

# Display the results
print("Payment Analytics Summary:")
payment_behavior.show(truncate=False)

Payment Analytics Summary:
+------------+-----------+------------------+------------------+-----------------------+
|payment_type|total_trips|average_revenue   |total_revenue     |percentage_contribution|
+------------+-----------+------------------+------------------+-----------------------+
|1           |15203      |17.555421298426573|266895.06999997917|67.4579580245818       |
|2           |7203       |13.508571428573047|97302.24000001166 |31.960775613435683     |
|3           |94         |15.858829787234018|1490.7299999999977|0.41709189333096686    |
|4           |37         |17.404324324324328|643.9600000000002 |0.16417446865155078    |
+------------+-----------+------------------+------------------+-----------------------+



### Task 10: Final Operational Dataset



In [ ]:
from pyspark.sql.functions import sum, avg, count, col

# Group by 'Borough' and calculate the required aggregate metrics
operational_dataset = clean_df_with_borough.groupBy("Borough") \
    .agg(
        count("*").alias("Total_Trips"),
        sum("total_amount").alias("Total_Revenue"),
        avg("total_amount").alias("Average_Revenue_per_Trip"),
        avg("trip_distance").alias("Average_Distance"),
        avg("trip_duration_minutes").alias("Average_Duration")
    ) \
    .orderBy(col("Total_Revenue").desc())

print("Summarized operational dataset created:")
operational_dataset.show(truncate=False)

Summarized operational dataset created:
+-------------+-----------+------------------+------------------------+------------------+------------------+
|Borough      |Total_Trips|Total_Revenue     |Average_Revenue_per_Trip|Average_Distance  |Average_Duration  |
+-------------+-----------+------------------+------------------------+------------------+------------------+
|Manhattan    |20507      |293290.699999942  |14.30197981176876       |2.3759209050568213|15.86567188439703 |
|Queens       |1367       |61479.07000000016 |44.973716166788705      |11.148119970738836|34.223945379175774|
|Unknown      |326        |5578.510000000022 |17.111993865030744      |3.031472392638033 |23.62888548057258 |
|Brooklyn     |312        |5308.000000000016 |17.012820512820564      |3.4224358974358995|14.212446581196579|
|Bronx        |16         |258.93000000000006|16.183125000000004      |3.8593750000000004|12.159375         |
|N/A          |7          |257.19            |36.74142857142857       |4.2271428

In [ ]:
# Define the output path for the Parquet file
output_path = "/content/operational_dataset.parquet"

# Store the result in Parquet format
operational_dataset.write.mode("overwrite").parquet(output_path)

print(f"Operational dataset saved to {output_path} in Parquet format.")

Operational dataset saved to /content/operational_dataset.parquet in Parquet format.
